# 02. Azure ML Pipeline Creation - AutoML for Images (Instance Segmentation)
This notebook demonstrates creation of an Azure ML pipeline designed to load a labeled image dataset from an Azure Machine Learning workspace, to submit an AutoML for Images run to train a new instance segmentation model, and to register that model into the workspace. <i>Run this notebook after running </i>`01_Setup_AML_Env.ipynb`.

### Import required packages

In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DEFAULT_GPU_IMAGE
from azureml.pipeline.core import Pipeline, PipelineParameter
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import PipelineParameter

import warnings
warnings.filterwarnings('ignore')

### Connect to AML workspace and get reference to GPU training compute

In [ ]:
ws = Workspace.from_config()
cluster_name = 'automlimagescompute'
compute_target = ComputeTarget(workspace=ws, name=cluster_name)

### Create Run Configuration
The `RunConfiguration` defines the environment used across all python steps. You can optionally add additional conda or pip packages to be added to your environment. [More details here](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.conda_dependencies.condadependencies?view=azure-ml-py).

Here, we also register the environment to the AML workspace so that it can be used for future retraining and inferencing operations.

In [ ]:
run_config = RunConfiguration()
run_config.environment.docker.base_image = DEFAULT_GPU_IMAGE
run_config.environment.python.conda_dependencies = CondaDependencies.create()
run_config.environment.python.conda_dependencies.add_conda_package("numpy==1.18.5")
run_config.environment.python.conda_dependencies.add_conda_package("libffi=3.3")
run_config.environment.python.conda_dependencies.set_pip_requirements([
    "azureml-core==1.41.0",
    "azureml-mlflow==1.41.0",
    "azureml-dataset-runtime==1.41.0",
    "azureml-telemetry==1.41.0",
    "azureml-responsibleai==1.41.0",
    "azureml-automl-core==1.41.0",
    "azureml-automl-runtime==1.41.0",
    "azureml-train-automl-client==1.41.0",
    "azureml-defaults==1.41.0",
    "azureml-interpret==1.41.0",
    "azureml-train-automl-runtime==1.41.0",
    "azureml-automl-dnn-vision==1.41.0",
    "azureml-dataprep>=2.24.4"
])
run_config.environment.python.conda_dependencies.set_python_version('3.7')
run_config.environment.name = "AutoMLForImagesEnv"
run_config.environment.register(ws)

### Define Pipeline Parameters
`PipelineParameter` objects serve as variable inputs to an Azure ML pipeline and can be specified at runtime. Below we define the following parameters for our Azure ML Pipeline:

| Parameter Name | Parameter Description |
|----------------|-----------------------|
| `model_name` | Name of the custom object detection model to be trained (used for model registration). |
| `dataset_name` | The name of the dataset to be used for instance segmentation model training within the pipeline. |

[PipelineParameter](https://docs.microsoft.com/en-us/python/api/azureml-pipeline-core/azureml.pipeline.core.graph.pipelineparameter?view=azure-ml-py)

In [ ]:
model_name = PipelineParameter(name='model_name', default_value='Street_Segmentation_Model')
dataset_name = PipelineParameter(name='dataset_name', default_value='TRAIN_AML_Labeled_Street_Images')

### Define Pipeline Steps
The pipeline below consists of a single step which executes an associated python script located in the `./pipeline_step_scripts` dir. In this step we call the script located at `./pipeline_step_scripts/automl_job.py` which retrieves an Image Dataset from the AML workspace (referenced by the `dataset_name` parameter and triggers execution of an AutoML for Images training job. Upon completion of this job, the trained model is automatically registered in the AML workspace according to the value provided in the `model_name` parameter.

<i>Note:</i> The AutoML configuration settings can be modified inline by editing the `automl_job.py` file. Additionally, certain fields can be added as `PipelineParameters` and passed into the executed python script step. Finally, advanced logic to perform A/B testing against newly trained models and historical best-performers can be integrated into this step (or a secondary step) to ensure the registered model is always the best performer.

In [ ]:
submit_job_step = PythonScriptStep(
    name='Submit AutoML Job',
    script_name='automl_job.py',
    arguments=[
        '--model_name', model_name,
        '--dataset_name', dataset_name,
        '--compute_name', cluster_name,
    ],
    compute_target=compute_target,
    source_directory='pipeline_step_scripts',
    allow_reuse=False,
    runconfig=run_config
)

### Create Pipeline
Pipelines are reusable in AML workflows that can be triggered in multiple ways (manual, programmatic, scheduled, etc.) Create an Azure ML Pipeline by specifying the pipeline steps to be executed.

[What are Machine Learning Pipelines?](https://docs.microsoft.com/en-us/azure/machine-learning/concept-ml-pipelines)

In [ ]:
pipeline = Pipeline(workspace=ws, steps=[submit_job_step])

### Create Published PipelineEndpoint
`PipelineEndpoints` can be used to create a versions of published pipelines while maintaining a consistent endpoint. These endpoint URLs can be triggered remotely by submitting an authenticated request and updates to the underlying pipeline are tracked in the AML workspace.

[PipelineEndpoint](https://docs.microsoft.com/en-us/python/api/azureml-pipeline-core/azureml.pipeline.core.pipeline_endpoint.pipelineendpoint?view=azure-ml-py)

In [ ]:
from azureml.pipeline.core import PipelineEndpoint

def published_pipeline_to_pipeline_endpoint(
    workspace,
    published_pipeline,
    pipeline_endpoint_name,
    pipeline_endpoint_description="AML Pipeline for training custom instance segmentation models using AutoML for Images.",
):
    try:
        pipeline_endpoint = PipelineEndpoint.get(
            workspace=workspace, name=pipeline_endpoint_name
        )
        print("using existing PipelineEndpoint...")
        pipeline_endpoint.add_default(published_pipeline)
    except Exception as ex:
        print(ex)
        # create PipelineEndpoint if it doesn't exist
        print("PipelineEndpoint does not exist, creating one for you...")
        pipeline_endpoint = PipelineEndpoint.publish(
            workspace=workspace,
            name=pipeline_endpoint_name,
            pipeline=published_pipeline,
            description=pipeline_endpoint_description
        )


pipeline_endpoint_name = 'Instance Segmentation Model Training'
pipeline_endpoint_description = 'Sample pipeline for training and registering a custom instance segmentation model'

published_pipeline = pipeline.publish(name=pipeline_endpoint_name,
                                     description=pipeline_endpoint_description,
                                     continue_on_step_failure=False)

published_pipeline_to_pipeline_endpoint(
    workspace=ws,
    published_pipeline=published_pipeline,
    pipeline_endpoint_name=pipeline_endpoint_name,
    pipeline_endpoint_description=pipeline_endpoint_description
)

### Optional: Trigger a Pipeline execution from the notebook
You can create an Experiment (logical collection for runs) and submit a pipeline run directly from this notebook by running the commands below.

In [ ]:
experiment = Experiment(ws, 'DEMO_AutoML_InstanceSegmentation')
run = experiment.submit(pipeline)